## Important Libraries

In [1]:
import dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

<ipython-input-1-d706ec01c456>:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-1-d706ec01c456>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
import numpy as np

## Read DataSet

In [3]:
train_df = pd.read_csv('train.csv')
train_df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Describing data

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## Preprocessing

In [5]:
train_df['Title'] = train_df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
train_df['Title'] = train_df['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
        'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

train_df['Title'] = train_df['Title'].replace('Mlle', 'Miss')
train_df['Title'] = train_df['Title'].replace('Ms', 'Miss')
train_df['Title'] = train_df['Title'].replace('Mme', 'Mrs')

In [6]:
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)

In [7]:
missing_train = pd.DataFrame()
missing_train['num of missed'] = train_df.isna().sum().sort_values(ascending=False)
missing_train['percent'] = train_df.isna().sum()/train_df.isna().count()
missing_train

,num of missed,percent
Cabin,687,0.771044
Age,177,0.198653
Embarked,2,0.002245
Title,0,0.000000
Fare,0,0.000000
Ticket,0,0.000000
Parch,0,0.000000
SibSp,0,0.000000
Sex,0,0.000000
Pclass,0,0.000000


In [8]:
# train_df['Age'].mean()

In [9]:
train_df['Age'] = train_df['Age'].fillna(30)

In [10]:
# train_df['Embarked'].value_counts()

In [11]:
train_df['Embarked'] = train_df['Embarked'].fillna('S')

In [12]:
train_df.drop('Cabin',axis=1,inplace=True)

In [13]:
total_People=len(train_df)
total_Survived = train_df['Survived'].sum()
total_UnSurvived = total_People - total_Survived

In [14]:
train_df.loc[ train_df['Age'] <= 10, 'AgeBand'] = 10
train_df.loc[(train_df['Age'] > 10) & (train_df['Age'] <= 20), 'AgeBand'] = 20
train_df.loc[(train_df['Age'] > 20) & (train_df['Age'] <= 30), 'AgeBand'] = 30
train_df.loc[(train_df['Age'] > 30) & (train_df['Age'] <= 40), 'AgeBand'] = 40
train_df.loc[(train_df['Age'] > 40) & (train_df['Age'] <= 50), 'AgeBand'] = 50
train_df.loc[(train_df['Age'] > 50) & (train_df['Age'] <= 60), 'AgeBand'] = 60
train_df.loc[(train_df['Age'] > 60) & (train_df['Age'] <= 70), 'AgeBand'] = 70
train_df.loc[(train_df['Age'] > 70) & (train_df['Age'] <= 80), 'AgeBand'] = 80
train_df.loc[ train_df['Age'] > 80, 'AgeBand']=90

In [15]:
sorted_list=sorted([int(AgeBand) for AgeBand in train_df['AgeBand'].unique()])
dic_mark={str(AgeBand):str(AgeBand) for AgeBand in sorted_list}

In [16]:
import plotly.graph_objects as go
Sex_stat=train_df.groupby(by='Sex', as_index=False)['Survived'].sum()
Cla_stat = train_df[train_df['Sex']=='female'].groupby(by='Pclass', as_index=False)['Survived'].sum()
PieFig = go.Figure(go.Sunburst(
        labels=["Global Cases", "Global Deaths", "Global Survived",'Fmale','Male'],
        parents=["", "Global Cases", "Global Cases","Global Survived","Global Survived"],
        values=[total_People, total_UnSurvived, total_Survived,Sex_stat['Survived'][0],Sex_stat['Survived'][1]],
        branchvalues='total',
        marker=dict(
            colorscale='RdBu'),
        hovertemplate='<b>%{label} </b> <br> Sales: %{value}<br> Success rate: %{color:.2f}'
    
        ))

In [17]:
app = JupyterDash(external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css'])

In [25]:
app.layout = html.Div([
    html.Div([
            html.Div([
                html.H3("Titanic", style={'textAlign': 'center',"marginBottom": "0px", 'color': 'white','fontSize': 40}),
                html.H5("Titanic - Dashboard", style={'textAlign': 'center',"marginTop": "0px", 'color': 'white'}),
            ])
        ], className="six column", id="title"),
  
    #################################################################################################################
    html.Div([
        html.Div([
            html.H6(children='Global Cases',
                    style={
                        'textAlign': 'center',
                        'color': 'white',
                        'fontSize': 40 }
                    ),

            html.P(f"{total_People:,.0f}",
                   style={
                       'textAlign': 'center',
                       'color': 'orange',
                       'fontSize': 40}
                   ),

            ], className="card_container four columns",
            style={"background": "#1f2c56", "margin":16 , "padding":15
                  ,"boxShadow": "2px 2px 2px #1f2c56"}
        ),

        html.Div([
            html.H6(children='Global Deaths',
                    style={
                        'textAlign': 'center',
                        'color': 'white',
                        'fontSize':40}
                    ),

            html.P(f"{total_UnSurvived:,.0f}",
                   style={
                       'textAlign': 'center',
                       'color': '#dd1e35',
                       'fontSize': 40}
                   ),

            ], className="card_container four columns",
            style={"background": "#1f2c56", "margin":16 , "padding":15
                  ,"boxShadow": "2px 2px 2px #1f2c56"}
        ),
        html.Div([
            html.H6(children='Global Survived',
                    style={
                        'textAlign': 'center',
                        'color': 'white',
                        'fontSize':40 }
                    ),

            html.P(f"{total_Survived:,.0f}",
                   style={
                       'textAlign': 'center',
                       'color': 'green',
                       'fontSize': 40}
                   ),

            ], className="card_container four columns"
            ,style={"background": "#1f2c56", "margin":16 , "padding":15
                  ,"boxShadow": "2px 2px 2px #1f2c56"}
        )

    ], className="row flexDisplay"),
    
    ############################################################################################################
     html.Div([
         
         html.Div(children=[
             
             dcc.Graph(id='ScatterGraph'),
             html.Div([
                 dcc.Slider(id='my-slider',min=train_df['Age'].min(),max=train_df['Age'].max(),\
                           marks=dic_mark,\
                            value=train_df['Age'].max())
                 ],style={'marginTop':40})
             
         ],className="create_container six columns"
             ,style={"background": "#1f2c56", "margin":16 , "padding":15
                  ,"boxShadow": "2px 2px 2px #1f2c56",'textAlign': 'center','color': 'white'}),
         
         html.Div([
             
             html.P('Select Survived Or Not:', className='fix_label',  style={'textAlign': 'center','color': 'white','marginBottom':10}),
             dcc.Dropdown(
                    id='drop_down_Sur',
                    options=[
                        {'label': 'Survived', 'value': 1 } ,
                        { 'label': 'NotSurvived', 'value': 0}
                    ],
                    value=1,
                    placeholder='choose Status...'),
             
             html.Div([dcc.Graph(id='AgeFig')],style={'marginTop':10})
             
         ],className="create_container six columns"
             ,style={"background": "#1f2c56", "margin":16 , "padding":15
                  ,"boxShadow": "2px 2px 2px #1f2c56"}),
         
         
     ], className="row flexDisplay"),
    
    ############################################################################################################
    
     html.Div([
         
         html.Div([
             
             html.P('Classified By:', className='fix_label', style={'textAlign': 'center',
                                                                             'color': 'white',
                                                                             'marginBottom':10}),
             dcc.Dropdown(
                    id='drop_down_Class',
                    options=[
                        {'label': 'Pclass', 'value': 'Pclass' } ,
                        { 'label': 'Gender', 'value': 'Sex'},
                        { 'label': 'Embarked', 'value': 'Embarked'}
                    ],
                    value='Pclass',
                    placeholder='choose Status...'),
             
             html.Div([dcc.Graph(id='ClassFig')],style={'marginTop':10})
             
         ],className="create_container six columns"
             ,style={"background": "#1f2c56", "margin":16 , "padding":15
                  ,"boxShadow": "2px 2px 2px #1f2c56"}),
         
         html.Div([
             
             html.P('Select Survived Or Not:', className='fix_label',  style={'textAlign': 'center','color': 'white','marginBottom':10}),
             dcc.Dropdown(
                    id='drop_down_Sur1',
                    options=[
                        {'label': 'Survived', 'value': 1 } ,
                        { 'label': 'NotSurvived', 'value': 0}
                    ],
                    value=1,
                    placeholder='choose Status...'),
             
             html.Div([dcc.Graph(id='TitleFig')],style={'marginTop':10})
             
         ],className="create_container six columns"
             ,style={"background": "#1f2c56", "margin":16 , "padding":15
                  ,"boxShadow": "2px 2px 2px #1f2c56"}),
           
         
         
     ], className="row flexDisplay"),
    html.Div([
         html.H3('Global Information', className='fix_label', style={'textAlign': 'center',
                                                                             'color': 'white',
                                                                             'marginBottom':10}),
        html.Div([dcc.Graph(figure=PieFig)],className="create_container five columns",style={"background": "#1f2c56", "margin":16 , "padding":15
                  ,"boxShadow": "2px 2px 2px #1f2c56","marginLeft":500})
        
    ], className="row flexDisplay")
    
    
], id="mainContainer",
    style={"display": "flex", "flexDirection": "column" , "background": "#192444"})

In [19]:
@app.callback(
    Output('ScatterGraph', 'figure'),
    Input('my-slider', 'value')
)
def update_My_Div(input_value1):
    filter_1_df = train_df[train_df.Age<=input_value1]
    fig = px.scatter(filter_1_df,x='Age',y='Fare',text='Pclass',color='Survived',size=filter_1_df['Pclass']*15,\
                     template='simple_white',title='Fare vs Age with Survivability and Pclass')
    return fig

In [20]:
@app.callback(
    Output('AgeFig', 'figure'),
    Input('drop_down_Sur', 'value')
)
def update_My_AgeDiv(input_value1):
    filter_1_df = train_df[train_df['Survived']==input_value1]
    fig=px.histogram(filter_1_df,x='Age',nbins=40,\
                         title='Age Distribution',histnorm='percent',template='simple_white')
    return fig
    

In [21]:
title_dic = {'Pclass':'Survivability based on  Pclass','Sex':'Survived based on  Gender',\
                 'Embarked':'Survived based on  Embarked'}

In [22]:
@app.callback(
    Output('ClassFig', 'figure'),
    Input('drop_down_Class', 'value')
)
def update_My_AgeDiv(input_value1):
    numOfClass = train_df.groupby(by=input_value1, as_index=False)['Survived'].sum()
    fig=px.bar(x=numOfClass[numOfClass.columns[0]],y=numOfClass[numOfClass.columns[1]],
               color=numOfClass[input_value1],template='simple_white',title=title_dic[input_value1])
    return fig

In [23]:
@app.callback(
    Output('TitleFig', 'figure'),
    Input('drop_down_Sur1', 'value')
)
def update_My_titleDiv(input_value1):
    filter_1_df = train_df[train_df['Survived']==input_value1]
    fig=px.histogram(filter_1_df,x='Title',nbins=40,\
                         title='Title Distribution',histnorm='percent',template='simple_white')
    return fig

In [26]:
app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
